### DATA PREPROGRESS

In [1]:
import numpy as np
import pandas as pd

def one_hot_encoding(arr):
    s = set()
    _sd = dict()
    for e in arr:
        s.add(e)

    sl = list(s)
    for i in range(len(sl)):
        encode = np.zeros(len(sl))
        encode[i] = 1
        _sd[sl[i]] = encode

    encoded_data = []
    for k in range(len(arr)):
        encoded_data.append(_sd[arr[k]])

    return np.array(encoded_data)


def data_preprocess(filepath):
    df = pd.read_excel(filepath)
    data = df.to_numpy()

    ip = data[:, 2]

    encoded_ip = one_hot_encoding(ip)

    # print(encoded_ip)

    data[:, 2] = encoded_ip.tolist()

    # print(data)
    expanded_data = []
    for row in data:
        expanded_data.append(np.concatenate([row[:2], row[2], row[3:]]).tolist())

    expanded_data = np.array(expanded_data)
    
    # for i in range(len(expanded_data)):
    #     expanded_data[i][0] = expanded_data[i][0] * 1000

    training_x = expanded_data[:, 1:]
    training_y = expanded_data[:, 0]

    return training_x, training_y


# Data preprocessing
file_path = "D:\\model_fit\\training\\training_data\\output2.xlsx"
training_x, training_y = data_preprocess(file_path)

for i in range(len(training_x)):
    print(training_x[i], training_y[i])

[6.21021000e+05 1.00000000e+00 0.00000000e+00 0.00000000e+00
 1.85840708e+01 2.07547170e+01 1.80180180e+01] 0.1475279331207275
[2.059368e+06 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00] 1.15568208694458
[1.08196200e+06 0.00000000e+00 1.00000000e+00 0.00000000e+00
 1.02040816e+00 2.02020202e+00 1.51515152e+00] 0.7066545486450195
[1.60912200e+06 0.00000000e+00 1.00000000e+00 0.00000000e+00
 1.01010101e+00 0.00000000e+00 5.07614213e-01] 1.612224340438843
[2.72355500e+06 0.00000000e+00 1.00000000e+00 0.00000000e+00
 1.28205128e+00 2.01005025e+00 6.45161290e-01] 1.999531507492065
[3.62115000e+06 0.00000000e+00 1.00000000e+00 0.00000000e+00
 1.83246073e+01 1.61290323e+01 1.85185185e+01] 5.916099071502686
[3.11404900e+06 0.00000000e+00 1.00000000e+00 0.00000000e+00
 9.87654321e+00 1.00000000e+01 9.87654321e+00] 3.237768888473511
[4.312771e+06 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00] 8.06132698059082
[3.246187e+06 0.0000

### MODEL COMPILE & FIT

In [2]:
### Data preprocessing using MinMaxScaler
# create MinMaxScaler Object

from sklearn.preprocessing import MinMaxScaler

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()


# scaled_training_x = training_x[:, 0].reshape(-1, 1)
# scaled_training_y = training_y[:].reshape(-1, 1)

# training_x[:, 0] = scaled_training_x[:, 0]
# training_y = scaled_training_y

# Define the split points based on time
train_size = int(len(training_x) * 0.8)
val_size = int(len(training_x) * 0.1)

# Split the data
x_train = training_x[:train_size]
y_train = training_y[:train_size]

x_val = training_x[train_size : train_size + val_size]
y_val = training_y[train_size : train_size + val_size]


# test data from other file
x_test = training_x[train_size + val_size :]
y_test = training_y[train_size + val_size :]

# x_train_first_column = x_train[:, 0].reshape(-1, 1)
# scaled_x_train_first_column = x_scaler.fit_transform(x_train_first_column)
# x_train[:, 0] = scaled_x_train_first_column.flatten()
# y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))  # reshap to 2D

# make the same sacling on valid data set and test data set
# x_val_scaled = x_scaler.transform(x_val)
# x_val_first_column = x_val[:, 0].reshape(-1, 1)
# scaled_x_val_first_column = x_scaler.fit_transform(x_val_first_column)
# x_val[:, 0] = scaled_x_val_first_column.flatten()
# y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1))  # reshap to 2D


# x_test_first_column = x_test[:, 0].reshape(-1, 1)
# scaled_x_test_first_column = x_scaler.fit_transform(x_test_first_column)
# x_test[:, 0] = scaled_x_test_first_column.flatten()
# y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))  # reshap to 2D

### NORMALIZATION

In [3]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input, Bidirectional, Flatten, TimeDistributed, Conv1D, MaxPooling1D, Conv2D
from keras.callbacks import Callback, EarlyStopping
from keras.optimizers import Adam


def create_sequences(dataset, time_steps):
    dataX, dataY = list(), list()
    for i in range(len(dataset)):
        endx = i + time_steps

        if endx > len(dataset) - 1:
            break

        dataset_x = dataset[i:endx, 1:]
        dataset_y = dataset[endx, 0]

        dataX.append(dataset_x)
        dataY.append(dataset_y)

    return np.array(dataX), np.array(dataY)

# Create sequences for each set
time_steps = 4


# x_test = training_x[train_size:]
# y_test = one_hot_encoding[train_size:]

# Create sequences for each set
x_train_sequences, y_train_sequences = create_sequences(x_train, time_steps)
x_val_sequences, y_val_sequences = create_sequences(x_val, time_steps)


# Initialize LossHistory with validation data
class LossHistory(Callback):
    def __init__(self, x_train, y_train, x_val, y_val):
        super().__init__()
        self.train_data = (x_train, y_train)
        self.validation_data = (x_val, y_val)
        self.losses = []
        self.val_losses = []
        self.train_errors = []
        self.val_errors = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs['loss'])
        self.val_losses.append(logs.get('val_loss'))
        self.train_errors.append(logs.get('mean_absolute_error'))
        self.val_errors.append(logs.get('val_mean_absolute_error'))


print(x_train_sequences)

[[[ 1.          0.          0.         18.5840708  20.75471698
   18.01801802]
  [ 0.          1.          0.          0.          0.
    0.        ]
  [ 0.          1.          0.          1.02040816  2.02020202
    1.51515152]
  [ 0.          1.          0.          1.01010101  0.
    0.50761421]]

 [[ 0.          1.          0.          0.          0.
    0.        ]
  [ 0.          1.          0.          1.02040816  2.02020202
    1.51515152]
  [ 0.          1.          0.          1.01010101  0.
    0.50761421]
  [ 0.          1.          0.          1.28205128  2.01005025
    0.64516129]]

 [[ 0.          1.          0.          1.02040816  2.02020202
    1.51515152]
  [ 0.          1.          0.          1.01010101  0.
    0.50761421]
  [ 0.          1.          0.          1.28205128  2.01005025
    0.64516129]
  [ 0.          1.          0.         18.32460733 16.12903226
   18.51851852]]

 ...

 [[ 0.          1.          0.          1.5         1.01522843
    1.50753769]
 

### FITTING

In [4]:
from keras.callbacks import TensorBoard
import os
import datetime

# log
log_dir = os.path.join("logs", "fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

def add_hidden_layer(
    model,
    units,
    layers,
    activation=None,
):
    if activation:
        if layers > 1:
            for layer in range(layers - 1):
                model.add(
                    LSTM(units=units, activation=activation, return_sequences=True)
                )
        model.add(LSTM(units=units, activation=activation, return_sequences=False))
    else:
        if layers > 1:
            for layer in range(layers - 1):
                model.add(LSTM(units=units, return_sequences=True))
        model.add(LSTM(units=units, return_sequences=False))


# loop params

LSTM_units = 512

Stop_patience = 20

Batch_size = 64

Learning_rate = 1e-2

Epochs = 500

Layers = 3

Activation = "relu"


# loss_function = 'mean_absolute_error'  # Assuming you are doing regression

# loss_function = 'categorical_crossentropy'

# loss_function = 'sparse_categorical_crossentropy'

loss_function = "mean_squared_error"


# Initialize LossHistory with validation data

history = LossHistory(
    x_train_sequences, y_train_sequences, x_val_sequences, y_val_sequences
)


def model_fit():

    print("epochs:", Epochs)

    print("batch:", Batch_size)

    print("units:", LSTM_units)

    # Build LSTM model

    model = Sequential()

    # Adding LSTM layer with L2 regularization
    model.add(Input(shape=(x_train_sequences.shape[1], x_train_sequences.shape[2])))
    # model.add(LSTM(units=LSTM_units, return_sequences=False))  # Regularization on the weights

    add_hidden_layer(
        model,
        LSTM_units,
        Layers,
        Activation
    )

    model.add(Dense(1))

    print("hidden layers:", Layers)

    # full con layer
    # model.add(Dense(units=16, activation=Activation))

    # Compile model

    model.compile(
        optimizer=Adam(learning_rate=Learning_rate),
        loss=loss_function,
        metrics=["mean_absolute_error"],
    )

    # Define early stopping

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=Stop_patience, restore_best_weights=True
    )

    # early_stopping = EarlyStopping(monitor='val_mean_absolute_error', patience=Stop_patience, restore_best_weights=True)

    # Train the model with early stopping

    model.fit(
        x_train_sequences,
        y_train_sequences,
        epochs=Epochs,
        batch_size=Batch_size,
        validation_data=(x_val_sequences, y_val_sequences),
        callbacks=[history, tensorboard_callback],
        # callbacks=[history, early_stopping, tensorboard_callback],
    )

    x_test_sequences, y_test_sequences = create_sequences(x_test, time_steps)

    prediction_results = model.predict(x_test_sequences)

    _abs = np.abs(y_test_sequences - prediction_results)

    mae = np.mean(_abs)

    print("MAE:", mae)

    for i in range(40):

        print(f"pred: {prediction_results[i]} | real: {y_test_sequences[i]}")


model_fit()

epochs: 500
batch: 64
units: 512
hidden layers: 3
Epoch 1/500
184/184 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - loss: 4446463459328.0000 - mean_absolute_error: 1716631.3750 - val_loss: 2124338823168.0000 - val_mean_absolute_error: 1260103.8750
Epoch 2/500
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - loss: 2150429753344.0000 - mean_absolute_error: 1258804.3750 - val_loss: 2116707811328.0000 - val_mean_absolute_error: 1249788.3750
Epoch 3/500
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - loss: 2223875555328.0000 - mean_absolute_error: 1271496.2500 - val_loss: 2080187351040.0000 - val_mean_absolute_error: 1255773.8750
Epoch 4/500
184/184 ━━━━━━━━━━━━━━━━━━━━ 11s 60ms/step - loss: 2108363243520.0000 - mean_absolute_error: 1245117.3750 - val_loss: 2099430555648.0000 - val_mean_absolute_error: 1257582.0000
Epoch 5/500
184/184 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - loss: 2084998479872.0000 - mean_absolute_error: 1242507.3750 - val_loss: 2202339377152.0000 - val_mean_absolute_error: 1278244.2500
Epoch

: 

### SAVE MODEL

In [ ]:
# Save the model
    # model.save("/content/drive/MyDrive/LSTM/predict_model_2_0v.keras")